In [1]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [5]:
snpeff = '/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/snpeff/20250863.annot'
annot = '/home/laura/DATABASES/Anotacion/COVID/resistance_covid.aa'

In [3]:
df = pd.read_csv(snpeff, sep="\t")

In [105]:
df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,Allele,Annotation,Annotation_Impact,Gene_Name,Gene_ID,Feature_Type,Feature_ID,Transcript_BioType,Rank,HGVS.c,HGVS.p,cDNA.pos / cDNA.length,CDS.pos / CDS.length,AA.pos / AA.length,ERRORS / WARNINGS / INFO,Codon_change,AA_change,DP,INFO,TEST
0,NC_045512.2,241,.,C,T,72.0,PASS,T,upstream_gene_variant,MODIFIER,ORF1ab,GU280_gp01,transcript,GU280_gp01,protein_coding,NaN,-25C>T,NaN,NaN,NaN,NaN,25.0,.-.,.-.,615,"CODON=.-.;AA=.-.;DP=615;ANN=T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|GU280_gp01|protein_coding||c.-25C>T|||||25|,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725297.1|protein_coding||c.-25C>T|||||25|WARNING_TRANSCRIPT_NO_STOP_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742608.1|protein_coding||c.-25C>T|||||25|WARNING_TRANSCRIPT_NO_STOP_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|GU280_gp01.2|protein_coding||c.-25C>T|||||25|,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725298.1|protein_coding||c.-565C>T|||||565|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742609.1|protein_coding||c.-565C>T|||||565|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725299.1|protein_coding||c.-2479C>T|||||2479|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742610.1|protein_coding||c.-2479C>T|||||2479|WARNING_TRANSCRIPT_NO_START_CODON,T|intergenic_region|MODIFIER|CHR_START-ORF1ab|CHR_START-GU280_gp01|intergenic_region|CHR_START-GU280_gp01|||n.241C>T||||||",
1,NC_045512.2,625,cds-YP_009724389.1,G,T,66.0,PASS,T,missense_variant,MODERATE,ORF1ab,GU280_gp01,transcript,GU280_gp01,protein_coding,1/2,360G>T,Lys120Asn,360/21291,360/21291,120/7096,NaN,AAG-AAT,K-N,311,"CODON=AAG-AAT;AA=K-N;DP=311;ANN=T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|GU280_gp01|protein_coding|1/2|c.360G>T|p.Lys120Asn|360/21291|360/21291|120/7096||,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|YP_009725297.1|protein_coding|1/1|c.360G>T|p.Lys120Asn|360/540|360/540|120/179||WARNING_TRANSCRIPT_NO_STOP_CODON,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|YP_009742608.1|protein_coding|1/1|c.360G>T|p.Lys120Asn|360/540|360/540|120/179||WARNING_TRANSCRIPT_NO_STOP_CODON,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|GU280_gp01.2|protein_coding|1/1|c.360G>T|p.Lys120Asn|360/13218|360/13218|120/4405||,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725298.1|protein_coding||c.-181G>T|||||181|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742609.1|protein_coding||c.-181G>T|||||181|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725299.1|protein_coding||c.-2095G>T|||||2095|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742610.1|protein_coding||c.-2095G>T|||||2095|WARNING_TRANSCRIPT_NO_START_CODON",
2,NC_045512.2,625,cds-YP_009725295.1,G,T,66.0,PASS,T,missense_variant,MODERATE,ORF1ab,GU280_gp01,transcript,GU280_gp01,protein_coding,1/2,360G>T,Lys120Asn,360/21291,360/21291,120/7096,NaN,AAG-AAT,K-N,311,"CODON=AAG-AAT;AA=K-N;DP=311;ANN=T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|GU280_gp01|protein_coding|1/2|c.360G>T|p.Lys120Asn|360/21291|360/21291|120/7096||,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|YP_009725297.1|protein_coding|1/1|c.360G>T|p.Lys120Asn|360/540|360/540|120/179||WARNING_TRANSCRIPT_NO_STOP_CODON,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|YP_009742608.1|protein_coding|1/1|c.360G>T|p.Lys120Asn|360/540|360/540|120/179||WARNING_TRANSCRIPT_NO_STOP_CODON,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|GU280_gp01.2|protein_coding|1/1|c.360G>T|p.Lys120Asn|360/13218|360/13218|120/4405||,T|upstream_gene_variant

In [80]:
dfa = pd.read_csv(annot, sep="\t", names=['aa', 'annot'])

In [87]:
dfa

,aa,annot,aaAnnot
0,Lys417Glu,REGN10933,Lys417Glu:REGN10933
1,Lys444Gln,REGN10987,Lys444Gln:REGN10987
2,Val445Ala,REGN10987,Val445Ala:REGN10987
3,Tyr453Phe,REGN10933,Tyr453Phe:REGN10933
4,Leu455Phe,REGN10933,Leu455Phe:REGN10933
5,Phe486Val,REGN10933,Phe486Val:REGN10933
6,Gln493Lys,REGN10933,Gln493Lys:REGN10933
7,His655Tyr,REGN10987+REGN10933,His655Tyr:REGN10987+REGN10933
8,Arg682Gln,REGN10987+REGN10933,Arg682Gln:REGN10987+REGN10933
9,Arg685Ser,REGN10987,Arg685Ser:REGN10987


In [102]:
def checkAA(snpEffRow, dfAnnot):
    df = dfAnnot
    df['aaAnnot'] = df['aa'] + ":" + df['annot']
    presence_list = [annot in snpEffRow for annot in dfAnnot.aa]
    annotation_list = np.array(df.aaAnnot.tolist())
    return (',').join(annotation_list[np.array(presence_list)])


In [103]:
df['TEST'] = df.apply(lambda x: checkAA(x.INFO, dfa), axis=1)


In [104]:
df[['#CHROM', 'POS', 'TEST']]

,#CHROM,POS,TEST
0,NC_045512.2,241,
1,NC_045512.2,625,
2,NC_045512.2,625,
3,NC_045512.2,1059,
4,NC_045512.2,1059,
5,NC_045512.2,1060,
6,NC_045512.2,1060,
7,NC_045512.2,1349,
8,NC_045512.2,1349,
9,NC_045512.2,3037,
